# Soluciones a los ejercicios de Machine Learning con Scikit-Learn
Este notebook contiene las soluciones de referencia para los ejercicios **E1–E5**.
Incluye clasificación, regresión, reducción de dimensionalidad, clustering y generación de datos sintéticos.


## E1: Clasificación con el dataset de vinos

In [ ]:

import pandas as pd
import numpy as np
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar dataset
wine = datasets.load_wine()
X, y = wine.data, wine.target

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenamiento con SVM
model = LinearSVC(max_iter=5000)
model.fit(X_train, y_train)

# Predicciones
y_pred = model.predict(X_test)

# Métricas
print(classification_report(y_test, y_pred, target_names=wine.target_names))

# Matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap="Blues", xticklabels=wine.target_names, yticklabels=wine.target_names)
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.show()


## E2: Regresión con el dataset de precios de Boston

In [ ]:

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Cargar dataset
boston = load_boston()
X, y = boston.data, boston.target

# División train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo de regresión lineal
reg = LinearRegression()
reg.fit(X_train, y_train)

# Predicciones
y_pred = reg.predict(X_test)

# Métricas
print("MSE:", mean_squared_error(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R²:", r2_score(y_test, y_pred))

# Gráfico real vs predicho
plt.scatter(y_test, y_pred, alpha=0.7)
plt.xlabel("Valores reales")
plt.ylabel("Predicciones")
plt.title("Regresión Lineal - Boston Housing")
plt.show()


## E3: Reducción de dimensionalidad con PCA

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Normalización
scaler = StandardScaler()
X_scaled = scaler.fit_transform(wine.data)

# PCA a 2 componentes
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Visualización
plt.figure(figsize=(8,6))
plt.scatter(X_pca[:,0], X_pca[:,1], c=wine.target, cmap="viridis", alpha=0.7)
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.title("PCA con dataset Wine")
plt.colorbar(label="Clase")
plt.show()


## E4: Modelos de clusterización

In [ ]:

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

X = wine.data

# KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
labels_kmeans = kmeans.fit_predict(X)
print("Silhouette Score (KMeans):", silhouette_score(X, labels_kmeans))

# DBSCAN
dbscan = DBSCAN(eps=1.5, min_samples=5)
labels_dbscan = dbscan.fit_predict(X)
if len(set(labels_dbscan)) > 1:
    print("Silhouette Score (DBSCAN):", silhouette_score(X, labels_dbscan))
else:
    print("DBSCAN no formó clusters útiles")

# Visualización con PCA
X_pca = PCA(n_components=2).fit_transform(X)
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_kmeans, cmap="viridis", alpha=0.7)
plt.title("Clusters KMeans")

plt.subplot(1,2,2)
plt.scatter(X_pca[:,0], X_pca[:,1], c=labels_dbscan, cmap="plasma", alpha=0.7)
plt.title("Clusters DBSCAN")

plt.show()


## E5: Creación de un juego de datos sintéticos

In [ ]:

import numpy as np
import pandas as pd

# Número de individuos
N = 100000

# Generar sexo
sexo = np.random.choice(["Hombre","Mujer"], size=N, p=[0.49, 0.51])

# Estatura y peso en función del sexo
estatura = np.where(sexo=="Hombre",
                    np.random.normal(179.3, 7.0, N),
                    np.random.normal(176.3, 6.6, N))

peso = np.where(sexo=="Hombre",
                np.random.normal(78.9, 11.8, N),
                np.random.normal(60.4, 9.7, N))

# Calcular BMI
bmi = peso / (estatura/100)**2

# DataFrame
df = pd.DataFrame({"Sexo":sexo, "Estatura":estatura, "Peso":peso, "BMI":bmi})
df.head()


In [ ]:

# Definir categorías de BMI
def clasificar_bmi(x):
    if x < 18.5:
        return "Bajo peso"
    elif x < 25:
        return "Normal"
    elif x < 30:
        return "Sobrepeso"
    else:
        return "Obesidad"

df["Categoria_BMI"] = df["BMI"].apply(clasificar_bmi)
df["Categoria_BMI"].value_counts(normalize=True)


In [ ]:

from sklearn.linear_model import LinearRegression, LogisticRegression

# Modelo de regresión para predecir BMI
X = df[["Peso","Estatura"]]
y = df["BMI"]

reg = LinearRegression()
reg.fit(X, y)
print("Coeficientes regresión:", reg.coef_)
print("Intercept:", reg.intercept_)

# Modelo de clasificación: ¿tiene sobrepeso o no?
df["Sobrepeso"] = (df["BMI"] >= 25).astype(int)
X = df[["Peso","Estatura"]]
y = df["Sobrepeso"]

clf = LogisticRegression(max_iter=500)
clf.fit(X, y)
print("Precisión clasificación sobrepeso:", clf.score(X, y))
